In [26]:
import os
from dotenv import load_dotenv
load_dotenv()
openai_key = os.getenv("OPEN_AI_KEY")
from openai import OpenAI
from pydantic import BaseModel
from typing import Literal, Optional, Any

client = OpenAI(api_key=openai_key)

class OutputSchemaPurpose(BaseModel):
    """Schema for Gemini's classification output."""
    purpose: Literal["pre_authorization", "other"]
    name: Optional[str]
    payer: Optional[Literal["cohere", "humana"]]

class PurposeClass(BaseModel):
    items: list[OutputSchemaPurpose]

SYSTEM_PROMPT = """
You are an intent classification assistant for a US healthcare automation system.
Your task:
1. Determine if the user's input is about "pre_authorization" or "other".
2. If it is "pre_authorization", extract:
   - The person's name.
   - The payer (insurance partner), only from: ["cohere", "humana"].
3. If information is missing, leave the field as null.
4. Be concise and always follow the output JSON schema exactly.
"""

# def openai_function(user_input: str):
user_input = "hi"
response = client.responses.parse(
    model="gpt-4.1-mini",
    input=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_input},
    ],
    text_format=OutputSchemaPurpose,  # your Pydantic schema
)

parsed = response.output_parsed
purpose, name, payer = parsed.purpose, parsed.name, parsed.payer

In [27]:
print(purpose)
print(name)
print(payer)

other
None
None


In [4]:
import numpy  as np
from prompts import Greeetings

print(np.random.choice(Greeetings))

hello, hope you doing well, how can i help you today?


In [1]:
from table_files import PAYERS
def get_payer_details(payer_name: str) -> dict | None:
    """Fetch payer details from the mocked PAYERS table."""
    for payer in PAYERS:
        if payer["payer_name"].lower() == payer_name.lower():
            return payer
    return None
get_payer_details("cohere")

{'payer_id': 1,
 'payer_name': 'cohere',
 'payer_url': 'https://portal.cohere.com',
 'payer_credentials': {'userid': 'abc', 'password': '123'}}

In [5]:
from helpers import pre_authorization_workflow
def enrich_pre_authorization_response(patient_id: str, payer: str) -> dict:
    """
    Calls the pre_authorization_workflow() API and enriches its response 
    with payer details from get_payer_details().
    """
    # Step 1: Call workflow API
    workflow_data = pre_authorization_workflow(patient_id, payer)

    # Step 2: Fetch payer details
    payer_details = get_payer_details(payer)

    # Step 3: Enrich response
    enriched_response = {
        "payer_details": payer_details,
        "workflow_data": workflow_data
    }
    return enriched_response

enrich_pre_authorization_response("hjbh", "cohere")

{'payer_details': {'payer_id': 1,
  'payer_name': 'cohere',
  'payer_url': 'https://portal.cohere.com',
  'payer_credentials': {'userid': 'abc', 'password': '123'}},
 'workflow_data': {'response': [{'requestid': '4473',
    'appointmentid': '158231849-1',
    'appointmentdate': '10/27/2025 6:00:00 PM',
    'caseid': '12345873',
    'visittype': 'Subsequent',
    'authorizationtype': 'Initial',
    'clientspecialty': 'Radiology',
    'providerfirstname': 'JANE',
    'providerlastname': 'DOE',
    'providernpi': '1111111111',
    'providershortcode': '',
    'providerphone': '',
    'provideraddress1': 'TEST ADDRESS',
    'provideraddress2': '',
    'providercity': 'TEST CITY',
    'providerstate': 'NY',
    'providerzip': '10510',
    'providercountry': 'USA',
    'subscriberfirstname': 'JOHN',
    'subscriberlastname': 'DOE',
    'subscriberaddress1': 'TEST',
    'subscriberaddress2': '',
    'subscribercity': 'TEST',
    'subscriberstate': 'NY',
    'subscriberzip': '10562',
    'subs